In [ ]:
val program_tivo_df = spark.read.option("header","true").csv("tivo/ProgramMetadata.csv")
val imdb_title_df = spark.read.option("header","true").option("delimiter","\t").csv("imdb/title_akas.tsv")
val imdb_title_basics_df = spark.read.option("header","true").option("delimiter","\t").csv("imdb/title_basics.tsv")
val imdb_title_ratings_df = spark.read.option("header","true").option("delimiter","\t").csv("imdb/title_ratings.tsv")
val imdb_title_principals_df = spark.read.option("header","true").option("delimiter","\t").csv("imdb/title_principals.tsv")
val imdb_names_df = spark.read.option("header","true").option("delimiter","\t").csv("imdb/name_basics.tsv")


Intitializing Scala interpreter ...

In [128]:
println(" Schema for program_tivo_df")
program_tivo_df.printSchema()
println(" Schema for imdb_title_df")
imdb_title_df.printSchema()
println(" Schema for imdb_title_basics_df")
imdb_title_basics_df.printSchema()
println(" Schema for imdb_title_ratings_df")
imdb_title_ratings_df.printSchema()
println(" Schema for imdb_title_principals_df")
imdb_title_principals_df.printSchema()
println(" Schema for imdb_names_df")
imdb_names_df.printSchema()


 Schema for program_tivo_df
root
 |-- MASTER_TITLE: string (nullable = true)
 |-- EVENT_DATE: string (nullable = true)
 |-- CATEGORY_ID: string (nullable = true)
 |-- SUBCATEGORY_ID: string (nullable = true)
 |-- RUNTIME: string (nullable = true)
 |-- EPISODE_COUNT: string (nullable = true)
 |-- day: string (nullable = true)
 |-- week_text: string (nullable = true)
 |-- week_encoded: string (nullable = true)
 |-- month: string (nullable = true)
 |-- vwp_days: string (nullable = true)
 |-- weekend: string (nullable = true)
 |-- daypart: string (nullable = true)
 |-- weekOfTheYear: string (nullable = true)
 |-- EVENT_END_DATE: string (nullable = true)
 |-- airStartDate: string (nullable = true)
 |-- airStartTime: string (nullable = true)
 |-- airEndDate: string (nullable = true)
 |-- airEndTime: string (nullable = true)
 |-- AirYear: string (nullable = true)

 Schema for imdb_title_df
root
 |-- titleId: string (nullable = true)
 |-- ordering: string (nullable = true)
 |-- title: string (

In [129]:
program_tivo_df.createOrReplaceTempView("program_tivo_df")
imdb_title_df.createOrReplaceTempView("imdb_title_df")
imdb_title_basics_df.createOrReplaceTempView("imdb_title_basics_df")
imdb_title_ratings_df.createOrReplaceTempView("imdb_title_ratings_df")
imdb_title_principals_df.createOrReplaceTempView("imdb_title_principals_df")
imdb_names_df.createOrReplaceTempView("imdb_names_df")

In [130]:
val tmp_imdb_tivo_df = spark.sql(" SELECT * FROM (SELECT  i.title as MASTER_TITLE,titleId as imdb_program_id,COALESCE(b.genres,'') as imdb_genres,COALESCE(b.titleType,'') as imdb_titleType,row_number() over(PARTITION BY title ORDER BY startYear desc) as rank from imdb_title_df i join program_tivo_df p on lower(trim(i.title))=lower(trim(p.MASTER_TITLE)) join imdb_title_basics_df b ON b.tconst = i.titleId and titleType = 'tvSeries' and startYear <= 2018) where rank = 1")
tmp_imdb_tivo_df.createOrReplaceTempView("tmp_imdb_tivo_df")
println(tmp_imdb_tivo_df.select('MASTER_TITLE).distinct.count())
tmp_imdb_tivo_df.printSchema()




262
root
 |-- MASTER_TITLE: string (nullable = true)
 |-- imdb_program_id: string (nullable = true)
 |-- imdb_genres: string (nullable = false)
 |-- imdb_titleType: string (nullable = false)
 |-- rank: integer (nullable = true)



tmp_imdb_tivo_df: org.apache.spark.sql.DataFrame = [MASTER_TITLE: string, imdb_program_id: string ... 3 more fields]


In [131]:
//print(tmp_imdb_tivo_df.count())
//tmp_imdb_tivo_df.createOrReplaceTempView("tmp_imdb_tivo_df")
//spark.sql("select distinct MASTER_TITLE from tmp_imdb_tivo_df").count()
spark.sql("select imdb_program_id,count(MASTER_TITLE) from tmp_imdb_tivo_df GROUP BY imdb_program_id HAVING count(MASTER_TITLE)>1").show(40,false)


+---------------+-------------------+
|imdb_program_id|count(MASTER_TITLE)|
+---------------+-------------------+
|tt3121722      |2                  |
|tt1596786      |3                  |
|tt4839610      |2                  |
|tt0206512      |2                  |
|tt3487356      |2                  |
|tt4858114      |2                  |
|tt6688750      |2                  |
|tt5531466      |2                  |
|tt1884856      |3                  |
|tt6549760      |2                  |
|tt2229129      |2                  |
+---------------+-------------------+



In [132]:
spark.sql("select master_title from tmp_imdb_tivo_df where imdb_program_id = 'tt6549760'").show(10,false)

spark.sql("SELECT * FROM imdb_title_df WHERE titleId = 'tt6549760'").show(10,false)

+---------------------+
|master_title         |
+---------------------+
|The Ollie & Moon Show|
|Ollie y Moon         |
+---------------------+

+---------+--------+---------------------+------+--------+-----------+----------+---------------+
|titleId  |ordering|title                |region|language|types      |attributes|isOriginalTitle|
+---------+--------+---------------------+------+--------+-----------+----------+---------------+
|tt6549760|1       |Ollie & Moon         |ES    |\N      |imdbDisplay|\N        |0              |
|tt6549760|2       |The Ollie & Moon Show|\N    |\N      |original   |\N        |1              |
|tt6549760|3       |The Ollie & Moon Show|FR    |\N      |\N         |\N        |0              |
|tt6549760|4       |Ollie y Moon         |US    |es      |alternative|\N        |0              |
+---------+--------+---------------------+------+--------+-----------+----------+---------------+



In [133]:
var imdb_tivo_df = spark.sql("SELECT p.*,imdb_program_id,imdb_genres,imdb_titleType from program_tivo_df p JOIN tmp_imdb_tivo_df t ON p.MASTER_TITLE = t.MASTER_TITLE")

imdb_tivo_df: org.apache.spark.sql.DataFrame = [MASTER_TITLE: string, EVENT_DATE: string ... 21 more fields]


In [134]:
imdb_tivo_df.printSchema()
imdb_tivo_df.createOrReplaceTempView("imdb_tivo_df")
imdb_tivo_df.persist()

root
 |-- MASTER_TITLE: string (nullable = true)
 |-- EVENT_DATE: string (nullable = true)
 |-- CATEGORY_ID: string (nullable = true)
 |-- SUBCATEGORY_ID: string (nullable = true)
 |-- RUNTIME: string (nullable = true)
 |-- EPISODE_COUNT: string (nullable = true)
 |-- day: string (nullable = true)
 |-- week_text: string (nullable = true)
 |-- week_encoded: string (nullable = true)
 |-- month: string (nullable = true)
 |-- vwp_days: string (nullable = true)
 |-- weekend: string (nullable = true)
 |-- daypart: string (nullable = true)
 |-- weekOfTheYear: string (nullable = true)
 |-- EVENT_END_DATE: string (nullable = true)
 |-- airStartDate: string (nullable = true)
 |-- airStartTime: string (nullable = true)
 |-- airEndDate: string (nullable = true)
 |-- airEndTime: string (nullable = true)
 |-- AirYear: string (nullable = true)
 |-- imdb_program_id: string (nullable = true)
 |-- imdb_genres: string (nullable = false)
 |-- imdb_titleType: string (nullable = false)



In [135]:
spark.sql("SELECT CATEGORY_ID,COUNT(*) FROM imdb_tivo_df GROUP BY CATEGORY_ID").show(20)

+-----------+--------+
|CATEGORY_ID|count(1)|
+-----------+--------+
|          3|      83|
|          8|      56|
|          5|     100|
|          4|       4|
+-----------+--------+



In [136]:
spark.sql("SELECT SUBCATEGORY_ID,CATEGORY_ID,COUNT(*) FROM imdb_tivo_df GROUP BY SUBCATEGORY_ID,CATEGORY_ID ORDER BY 2,1,3").show(25)

+--------------+-----------+--------+
|SUBCATEGORY_ID|CATEGORY_ID|count(1)|
+--------------+-----------+--------+
|            11|          3|      59|
|            12|          3|       7|
|            17|          3|      15|
|            23|          3|       2|
|            76|          4|       4|
|             1|          5|       5|
|            12|          5|      20|
|            14|          5|      10|
|            15|          5|      15|
|            23|          5|       9|
|            44|          5|      34|
|            48|          5|       1|
|            52|          5|       1|
|            54|          5|       1|
|            65|          5|       4|
|             2|          8|       3|
|            21|          8|      18|
|            27|          8|       5|
|            28|          8|      17|
|             4|          8|       2|
|            57|          8|       6|
|            74|          8|       4|
|            80|          8|       1|
+-----------

In [137]:
val imdb_tivo_df2 = spark.sql("SELECT i.*,COALESCE(averageRating,-9999) as imdb_averageRating,COALESCE(numVotes,-999999) as imdb_numVotes from imdb_tivo_df i left join imdb_title_ratings_df r on i.imdb_program_id = r.tconst")

imdb_tivo_df2: org.apache.spark.sql.DataFrame = [MASTER_TITLE: string, EVENT_DATE: string ... 23 more fields]


In [149]:
imdb_tivo_df2.printSchema()
imdb_tivo_df2.persist()
//print(imdb_tivo_df2.count())
//imdb_tivo_df2.createOrReplaceTempView("imdb_tivo_df2")

root
 |-- MASTER_TITLE: string (nullable = true)
 |-- EVENT_DATE: string (nullable = true)
 |-- CATEGORY_ID: string (nullable = true)
 |-- SUBCATEGORY_ID: string (nullable = true)
 |-- RUNTIME: string (nullable = true)
 |-- EPISODE_COUNT: string (nullable = true)
 |-- day: string (nullable = true)
 |-- week_text: string (nullable = true)
 |-- week_encoded: string (nullable = true)
 |-- month: string (nullable = true)
 |-- vwp_days: string (nullable = true)
 |-- weekend: string (nullable = true)
 |-- daypart: string (nullable = true)
 |-- weekOfTheYear: string (nullable = true)
 |-- EVENT_END_DATE: string (nullable = true)
 |-- airStartDate: string (nullable = true)
 |-- airStartTime: string (nullable = true)
 |-- airEndDate: string (nullable = true)
 |-- airEndTime: string (nullable = true)
 |-- AirYear: string (nullable = true)
 |-- imdb_program_id: string (nullable = true)
 |-- imdb_genres: string (nullable = false)
 |-- imdb_titleType: string (nullable = false)
 |-- imdb_averageRati

res120: imdb_tivo_df2.type = [MASTER_TITLE: string, EVENT_DATE: string ... 23 more fields]


In [150]:
val imdb_tivo_cast_df = spark.sql("SELECT imdb_program_id,p.* from imdb_tivo_df2 i LEFT JOIN imdb_title_principals_df p ON i.imdb_program_id = p.tconst")

imdb_tivo_cast_df: org.apache.spark.sql.DataFrame = [imdb_program_id: string, tconst: string ... 5 more fields]


In [151]:
imdb_tivo_cast_df.createOrReplaceTempView("imdb_tivo_cast_df")

In [145]:
//spark.sql("SELECT DISTINCT imdb_program_id FROM imdb_tivo_cast_df").count()

2019-06-22 17:34:53 ERROR RetryingBlockFetcher:143 - Exception while beginning fetch of 1 outstanding blocks 
java.io.IOException: Failed to connect to ban-1mthota-m-42e6:58287
	at org.apache.spark.network.client.TransportClientFactory.createClient(TransportClientFactory.java:245)
	at org.apache.spark.network.client.TransportClientFactory.createClient(TransportClientFactory.java:187)
	at org.apache.spark.network.netty.NettyBlockTransferService$$anon$2.createAndStart(NettyBlockTransferService.scala:113)
	at org.apache.spark.network.shuffle.RetryingBlockFetcher.fetchAllOutstanding(RetryingBlockFetcher.java:141)
	at org.apache.spark.network.shuffle.RetryingBlockFetcher.start(RetryingBlockFetcher.java:121)
	at org.apache.spark.network.netty.NettyBlockTransferService.fetchBlocks(NettyBlockTransferService.scala:123)
	at org.apache.spark.network.BlockTransferService.fetchBlockSync(BlockTransferService.scala:98)
	at org.apache.spark.storage.BlockManager.getRemoteBytes(BlockManager.scala:691)
	

2019-06-22 17:34:53 ERROR RetryingBlockFetcher:143 - Exception while beginning fetch of 1 outstanding blocks 
java.io.IOException: Failed to connect to ban-1mthota-m-42e6:58287
	at org.apache.spark.network.client.TransportClientFactory.createClient(TransportClientFactory.java:245)
	at org.apache.spark.network.client.TransportClientFactory.createClient(TransportClientFactory.java:187)
	at org.apache.spark.network.netty.NettyBlockTransferService$$anon$2.createAndStart(NettyBlockTransferService.scala:113)
	at org.apache.spark.network.shuffle.RetryingBlockFetcher.fetchAllOutstanding(RetryingBlockFetcher.java:141)
	at org.apache.spark.network.shuffle.RetryingBlockFetcher.start(RetryingBlockFetcher.java:121)
	at org.apache.spark.network.netty.NettyBlockTransferService.fetchBlocks(NettyBlockTransferService.scala:123)
	at org.apache.spark.network.BlockTransferService.fetchBlockSync(BlockTransferService.scala:98)
	at org.apache.spark.storage.BlockManager.getRemoteBytes(BlockManager.scala:691)
	

2019-06-22 17:35:03 WARN  TransportClientFactory:171 - DNS resolution for ban-1mthota-m-42e6:58287 took 4759 ms
2019-06-22 17:35:03 WARN  TransportClientFactory:171 - DNS resolution for ban-1mthota-m-42e6:58287 took 4710 ms
2019-06-22 17:35:03 WARN  TransportClientFactory:171 - DNS resolution for ban-1mthota-m-42e6:58287 took 4710 ms
2019-06-22 17:35:03 WARN  TransportClientFactory:171 - DNS resolution for ban-1mthota-m-42e6:58287 took 4710 ms
2019-06-22 17:35:03 ERROR RetryingBlockFetcher:143 - Exception while beginning fetch of 1 outstanding blocks (after 1 retries)
java.io.IOException: Failed to connect to ban-1mthota-m-42e6:58287
	at org.apache.spark.network.client.TransportClientFactory.createClient(TransportClientFactory.java:245)
	at org.apache.spark.network.client.TransportClientFactory.createClient(TransportClientFactory.java:187)
	at org.apache.spark.network.netty.NettyBlockTransferService$$anon$2.createAndStart(NettyBlockTransferService.scala:113)
	at org.apache.spark.networ

2019-06-22 17:35:03 ERROR RetryingBlockFetcher:143 - Exception while beginning fetch of 1 outstanding blocks (after 1 retries)
java.io.IOException: Failed to connect to ban-1mthota-m-42e6:58287
	at org.apache.spark.network.client.TransportClientFactory.createClient(TransportClientFactory.java:245)
	at org.apache.spark.network.client.TransportClientFactory.createClient(TransportClientFactory.java:187)
	at org.apache.spark.network.netty.NettyBlockTransferService$$anon$2.createAndStart(NettyBlockTransferService.scala:113)
	at org.apache.spark.network.shuffle.RetryingBlockFetcher.fetchAllOutstanding(RetryingBlockFetcher.java:141)
	at org.apache.spark.network.shuffle.RetryingBlockFetcher.lambda$initiateRetry$0(RetryingBlockFetcher.java:169)
	at java.util.concurrent.Executors$RunnableAdapter.call(Executors.java:511)
	at java.util.concurrent.FutureTask.run(FutureTask.java:266)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1142)
	at java.util.concurrent.ThreadPo

2019-06-22 17:35:08 ERROR RetryingBlockFetcher:143 - Exception while beginning fetch of 1 outstanding blocks (after 2 retries)
java.io.IOException: Failed to connect to ban-1mthota-m-42e6:58287
	at org.apache.spark.network.client.TransportClientFactory.createClient(TransportClientFactory.java:245)
	at org.apache.spark.network.client.TransportClientFactory.createClient(TransportClientFactory.java:187)
	at org.apache.spark.network.netty.NettyBlockTransferService$$anon$2.createAndStart(NettyBlockTransferService.scala:113)
	at org.apache.spark.network.shuffle.RetryingBlockFetcher.fetchAllOutstanding(RetryingBlockFetcher.java:141)
	at org.apache.spark.network.shuffle.RetryingBlockFetcher.lambda$initiateRetry$0(RetryingBlockFetcher.java:169)
	at java.util.concurrent.Executors$RunnableAdapter.call(Executors.java:511)
	at java.util.concurrent.FutureTask.run(FutureTask.java:266)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1142)
	at java.util.concurrent.ThreadPo

2019-06-22 17:35:42 WARN  BlockManager:87 - Failed to fetch block after 1 fetch failures. Most recent failure cause:
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:205)
	at org.apache.spark.network.BlockTransferService.fetchBlockSync(BlockTransferService.scala:115)
	at org.apache.spark.storage.BlockManager.getRemoteBytes(BlockManager.scala:691)
	at org.apache.spark.scheduler.TaskResultGetter$$anon$3$$anonfun$run$1.apply$mcV$sp(TaskResultGetter.scala:82)
	at org.apache.spark.scheduler.TaskResultGetter$$anon$3$$anonfun$run$1.apply(TaskResultGetter.scala:63)
	at org.apache.spark.scheduler.TaskResultGetter$$anon$3$$anonfun$run$1.apply(TaskResultGetter.scala:63)
	at org.apache.spark.util.Utils$.logUncaughtExceptions(Utils.scala:1991)
	at org.apache.spark.scheduler.TaskResultGetter$$anon$3.run(TaskResultGetter.scala:62)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1142)
	

	at org.apache.spark.scheduler.TaskResultGetter$$anon$3$$anonfun$run$1.apply(TaskResultGetter.scala:63)
	at org.apache.spark.scheduler.TaskResultGetter$$anon$3$$anonfun$run$1.apply(TaskResultGetter.scala:63)
	at org.apache.spark.util.Utils$.logUncaughtExceptions(Utils.scala:1991)
	at org.apache.spark.scheduler.TaskResultGetter$$anon$3.run(TaskResultGetter.scala:62)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1142)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:617)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.io.IOException: Failed to connect to ban-1mthota-m-42e6:58287
	at org.apache.spark.network.client.TransportClientFactory.createClient(TransportClientFactory.java:245)
	at org.apache.spark.network.client.TransportClientFactory.createClient(TransportClientFactory.java:187)
	at org.apache.spark.network.netty.NettyBlockTransferService$$anon$2.createAndStart(NettyBlockTransferService.scala:113)
	at org.apach

	at org.apache.spark.storage.BlockManager.getRemoteBytes(BlockManager.scala:691)
	at org.apache.spark.scheduler.TaskResultGetter$$anon$3$$anonfun$run$1.apply$mcV$sp(TaskResultGetter.scala:82)
	at org.apache.spark.scheduler.TaskResultGetter$$anon$3$$anonfun$run$1.apply(TaskResultGetter.scala:63)
	at org.apache.spark.scheduler.TaskResultGetter$$anon$3$$anonfun$run$1.apply(TaskResultGetter.scala:63)
	at org.apache.spark.util.Utils$.logUncaughtExceptions(Utils.scala:1991)
	at org.apache.spark.scheduler.TaskResultGetter$$anon$3.run(TaskResultGetter.scala:62)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1142)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:617)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.io.IOException: Failed to connect to ban-1mthota-m-42e6:58287
	at org.apache.spark.network.client.TransportClientFactory.createClient(TransportClientFactory.java:245)
	at org.apache.spark.network.client.TransportCl

org.apache.spark.SparkException:  Exception thrown in awaitResult:

In [ ]:
spark.sql("SELECT * FROM imdb_tivo_cast_df where imdb_program_id ='tt3487356' order by ordering").show(40,false)

In [ ]:
spark.sql("SELECT * FROM imdb_tivo_cast_df c join imdb_names_df n on n.nconst = c.nconst ").count()

In [ ]:
spark.sql("SELECT distinct imdb_program_id,category,characters,primaryName,knownForTitles FROM imdb_tivo_cast_df c join imdb_names_df n on n.nconst = c.nconst ").show(40,false)

In [229]:
val imdb_tivo_cast_tmp_df =  spark.sql("SELECT distinct imdb_program_id,CONCAT('\\\"cast\\\":{ \\\"category\\\": \\\"', category ,'\\\", \\\"PrimaryName\\\": \\\"', primaryName,'\\\", \\\"knownForTitles\\\": \\\"',knownForTitles, '\\\",\\\"characters\\\" : \\\"', replace(characters,'\\\"',' '), '\\\"}') as cast_information FROM imdb_tivo_cast_df c join imdb_names_df n on n.nconst = c.nconst ")

imdb_tivo_cast_tmp_df.createOrReplaceTempView("imdb_tivo_cast_tmp_df")
//imdb_tivo_cast_tmp_df.show(1,false)


imdb_tivo_cast_tmp_df: org.apache.spark.sql.DataFrame = [imdb_program_id: string, cast_information: string]


In [237]:
val imdb_tivo_cast_tmp2_df = spark.sql("SELECT imdb_program_id,CONCAT('{\\\"casts\\\":[ {',(CONCAT_WS('},{',COLLECT_LIST(cast_information))),'} ]}')as cast_details FROM imdb_tivo_cast_tmp_df GROUP BY imdb_program_id")

imdb_tivo_cast_tmp2_df: org.apache.spark.sql.DataFrame = [imdb_program_id: string, cast_details: string]


In [238]:
//imdb_tivo_cast_tmp2_df.filter('imdb_program_id=== "tt3487356").show(20,false)
imdb_tivo_cast_tmp2_df.createOrReplaceTempView("imdb_tivo_cast_tmp2_df")

In [239]:
val tivo_imdb_final_df = spark.sql("SELECT it.*, cast_details as imdb_cast_details FROM imdb_tivo_df2 it LEFT JOIN imdb_tivo_cast_tmp2_df itc on it.imdb_program_id = itc.imdb_program_id")

tivo_imdb_final_df: org.apache.spark.sql.DataFrame = [MASTER_TITLE: string, EVENT_DATE: string ... 24 more fields]


In [240]:
tivo_imdb_final_df.printSchema()

root
 |-- MASTER_TITLE: string (nullable = true)
 |-- EVENT_DATE: string (nullable = true)
 |-- CATEGORY_ID: string (nullable = true)
 |-- SUBCATEGORY_ID: string (nullable = true)
 |-- RUNTIME: string (nullable = true)
 |-- EPISODE_COUNT: string (nullable = true)
 |-- day: string (nullable = true)
 |-- week_text: string (nullable = true)
 |-- week_encoded: string (nullable = true)
 |-- month: string (nullable = true)
 |-- vwp_days: string (nullable = true)
 |-- weekend: string (nullable = true)
 |-- daypart: string (nullable = true)
 |-- weekOfTheYear: string (nullable = true)
 |-- EVENT_END_DATE: string (nullable = true)
 |-- airStartDate: string (nullable = true)
 |-- airStartTime: string (nullable = true)
 |-- airEndDate: string (nullable = true)
 |-- airEndTime: string (nullable = true)
 |-- AirYear: string (nullable = true)
 |-- imdb_program_id: string (nullable = true)
 |-- imdb_genres: string (nullable = false)
 |-- imdb_titleType: string (nullable = false)
 |-- imdb_averageRati

In [ ]:
tivo_imdb_final_df.count()

In [225]:
tivo_imdb_final_df.repartition(1).write.option("header","true").option("quote", "\u0000").csv("tivo_imdb/csv/version4/")

In [241]:
tivo_imdb_final_df.repartition(1).write.option("header","true").parquet("tivo_imdb/parquet/version5/")

In [242]:
val df = spark.read.parquet("tivo_imdb/parquet/version5/")
df.printSchema()
df.select('imdb_cast_details).show(5,false)

root
 |-- MASTER_TITLE: string (nullable = true)
 |-- EVENT_DATE: string (nullable = true)
 |-- CATEGORY_ID: string (nullable = true)
 |-- SUBCATEGORY_ID: string (nullable = true)
 |-- RUNTIME: string (nullable = true)
 |-- EPISODE_COUNT: string (nullable = true)
 |-- day: string (nullable = true)
 |-- week_text: string (nullable = true)
 |-- week_encoded: string (nullable = true)
 |-- month: string (nullable = true)
 |-- vwp_days: string (nullable = true)
 |-- weekend: string (nullable = true)
 |-- daypart: string (nullable = true)
 |-- weekOfTheYear: string (nullable = true)
 |-- EVENT_END_DATE: string (nullable = true)
 |-- airStartDate: string (nullable = true)
 |-- airStartTime: string (nullable = true)
 |-- airEndDate: string (nullable = true)
 |-- airEndTime: string (nullable = true)
 |-- AirYear: string (nullable = true)
 |-- imdb_program_id: string (nullable = true)
 |-- imdb_genres: string (nullable = true)
 |-- imdb_titleType: string (nullable = true)
 |-- imdb_averageRating

|{"casts":[ {"cast":{ "category": "self", "PrimaryName": "Adam Growe", "knownForTitles": "tt0770652,tt0481257,tt1594908,tt1312171","characters" : "[ Himself - Host ]"}} ]}                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

df: org.apache.spark.sql.DataFrame = [MASTER_TITLE: string, EVENT_DATE: string ... 24 more fields]


2019-06-22 21:43:30 WARN  HeartbeatReceiver:66 - Removing executor driver with no recent heartbeats: 563891 ms exceeds timeout 120000 ms
2019-06-22 21:43:30 ERROR TaskSchedulerImpl:70 - Lost executor driver on localhost: Executor heartbeat timed out after 563891 ms
2019-06-22 21:43:30 WARN  SparkContext:66 - Killing executors is not supported by current scheduler.
2019-06-22 21:43:30 WARN  BlockManagerMasterEndpoint:66 - No more replicas available for rdd_3146_97 !
2019-06-22 21:43:30 WARN  BlockManagerMasterEndpoint:66 - No more replicas available for taskresult_54965 !
2019-06-22 21:43:30 WARN  BlockManagerMasterEndpoint:66 - No more replicas available for rdd_3146_121 !
2019-06-22 21:43:30 WARN  BlockManagerMasterEndpoint:66 - No more replicas available for taskresult_57921 !
2019-06-22 21:43:30 WARN  BlockManagerMasterEndpoint:66 - No more replicas available for rdd_3146_2 !
2019-06-22 21:43:30 WARN  BlockManagerMasterEndpoint:66 - No more replicas available for rdd_3146_40 !
2019-

2019-06-22 21:43:30 WARN  BlockManagerMasterEndpoint:66 - No more replicas available for rdd_3146_0 !
2019-06-22 21:43:30 WARN  BlockManagerMasterEndpoint:66 - No more replicas available for rdd_3146_94 !
2019-06-22 21:43:30 WARN  BlockManagerMasterEndpoint:66 - No more replicas available for rdd_3146_199 !
2019-06-22 21:43:30 WARN  BlockManagerMasterEndpoint:66 - No more replicas available for broadcast_1631_piece0 !
2019-06-22 21:43:30 WARN  BlockManagerMasterEndpoint:66 - No more replicas available for rdd_3146_172 !
2019-06-22 21:43:30 WARN  BlockManagerMasterEndpoint:66 - No more replicas available for rdd_3146_68 !
2019-06-22 21:43:30 WARN  BlockManagerMasterEndpoint:66 - No more replicas available for rdd_3146_92 !
2019-06-22 21:43:30 WARN  BlockManagerMasterEndpoint:66 - No more replicas available for rdd_3146_157 !
2019-06-22 21:43:30 WARN  BlockManagerMasterEndpoint:66 - No more replicas available for rdd_3146_192 !
2019-06-22 21:43:30 WARN  BlockManagerMasterEndpoint:66 - No

2019-06-22 21:43:30 WARN  BlockManagerMasterEndpoint:66 - No more replicas available for rdd_3146_10 !
2019-06-22 21:43:30 WARN  BlockManagerMasterEndpoint:66 - No more replicas available for taskresult_57906 !
2019-06-22 21:43:30 WARN  BlockManagerMasterEndpoint:66 - No more replicas available for rdd_3146_75 !
2019-06-22 21:43:30 WARN  BlockManagerMasterEndpoint:66 - No more replicas available for taskresult_57916 !
2019-06-22 21:43:30 WARN  BlockManagerMasterEndpoint:66 - No more replicas available for rdd_3146_20 !
2019-06-22 21:43:30 WARN  BlockManagerMasterEndpoint:66 - No more replicas available for rdd_3146_149 !
2019-06-22 21:43:30 WARN  BlockManagerMasterEndpoint:66 - No more replicas available for rdd_3146_133 !
2019-06-22 21:43:30 WARN  BlockManagerMasterEndpoint:66 - No more replicas available for rdd_3146_140 !
2019-06-22 21:43:30 WARN  BlockManagerMasterEndpoint:66 - No more replicas available for rdd_3146_169 !
2019-06-22 21:43:30 WARN  BlockManagerMasterEndpoint:66 - N

In [ ]:
val wikidf = spark.read.parquet("")